In [1]:
import agent_factory
from plotter import plot_trend

2024-10-15 19:04:05.126375: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-15 19:04:05.133968: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-15 19:04:05.159226: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-15 19:04:05.199410: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-15 19:04:05.211278: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-15 19:04:05.242926: I tensorflow/core/platform/cpu_feature_gu

pygame 2.6.0 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
agent = agent_factory.Agent(lr=0.001, gamma=0.9, max_memory=50_000, batch_size=1024, visual=False)
train_result = agent.train_agent(N_GAME=100)
plot_trend(train_result)

# Prioritized Experience Replay

In [ ]:
agent = agent_factory.Agent_PER(lr=0.001, gamma=0.9, max_memory=50_000, batch_size=1024, visual=False)
train_result = agent.train_agent(N_GAME=100)
plot_trend(train_result)

# Double DQN

In [ ]:
agent = agent_factory.Agent_DoubleDQN(lr=0.001, gamma=0.9, max_memory=50_000, batch_size=1024, visual=False)
train_result = agent.train_agent(N_GAME=100)
plot_trend(train_result)

# PER + DDQN

epsilon greedy policy

In [ ]:
agent = agent_factory.Agent_DDQN_PER(lr=0.001, gamma=0.9, max_memory=50_000, batch_size=1024, visual=False)
train_result = agent.train_agent(N_GAME=100)
plot_trend(train_result)

softmax policy